<a href="https://colab.research.google.com/github/Sijie0607/-practicegithub/blob/main/behavior_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##6.26

In [13]:
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('/content/6.26 combined records - Sheet1.csv')

df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S') # change the timestamp into the python recognized format
# Filter rows where `from` column is `[Sponsor_Ad_Click_Counter]`
filtered_df = df[df['table_from'].isin(['[Sponsor_Ad_Click_Counter]', '[HCL_Tracking]', '[Web_Clicks]', '[User_Page_Tracking]', '[User_Other_Click_Activity]', 'Ads_Click_Count'])]

# Create a mask for rows to keep
mask = pd.Series([True] * len(df))

# Process each user_id
for user_id in filtered_df['user_id'].unique():
    # Filter rows for the current user_id
    user_data = filtered_df[filtered_df['user_id'] == user_id]

    # Find duplicate `click_activity` within the filtered data, where time is the same among all ad
    duplicates = user_data[user_data.duplicated(subset=['timestamp'], keep=False)]

    # Create a mask to exclude these rows from the original dataframe
    mask &= ~((df['user_id'] == user_id) & (df['timestamp'].isin(duplicates['timestamp'])))

# Apply the mask to the dataframe
df = df[mask]
df = df.reset_index(drop=True)
# Sort data by user_id and timestamp
data = df.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)
new_rows = []
#Insert the third party login activity before all third party ads activities
for user_id in data['user_id'].unique():
    user_data = data[data['user_id'] == user_id].sort_values(by='timestamp')
    for index, row in user_data.iterrows():
        if row['click_activities'] == 'ad click from partner eNewsletter':
            new_row = {
                'user_id': user_id,
                'table_from': '[User_Login_Tracking]',
                'click_action': 'login from third part eNewsletter',
                'timestamp': row['timestamp'] - pd.Timedelta(minutes=1),
                'click_activities': 'login from third party eNewsletter'
            }
            new_rows.append(new_row)
            same_time_rows = user_data[user_data['timestamp'] == row['timestamp']]
            for idx, same_time_row in same_time_rows.iterrows():
                new_rows.append({
                    'user_id': user_id,
                    'table_from': '[User_Login_Tracking]',
                    'click_action': 'login from third part eNewsletter',
                    'timestamp': same_time_row['timestamp'] - pd.Timedelta(seconds=50),
                    'click_activities': 'login from third party eNewsletter'
                })
        elif row['click_activities'] == 'ad click from ad partner website':
            new_row = {
                'user_id': user_id,
                'table_from': '[User_Login_Tracking]',
                'click_action': 'login from third part website',
                'timestamp': row['timestamp'] - pd.Timedelta(minutes=1),
                'click_activities': 'login from third party website'
            }
            new_rows.append(new_row)

new_rows_df = pd.DataFrame(new_rows)
data = pd.concat([data, new_rows_df], ignore_index=True)
data = data.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)


# Make sure before  Staff Directory there is a RIGHTSURE Web click
# Make sure there is an ‘ad click from eNewsletter’ before clicking on ‘cause we support’ and insurance products.
new_rows = []
for user_id in data['user_id'].unique():
    user_data = data[data['user_id'] == user_id].sort_values(by='timestamp')
    for index, row in user_data.iterrows():
        if row['click_activities'] == 'Staff Directory':
            if not user_data.loc[:index - 1, 'click_activities'].eq('RIGHTSURE homepage').any():
                new_row = {
                    'user_id': user_id,
                    'table_from': '[Web_Clicks]',
                    'click_action': 'https://www.rightsure.com/',
                    'timestamp': row['timestamp'] - pd.Timedelta(minutes=1),
                    'click_activities': 'RIGHTSURE homepage'
                }
                new_rows.append(new_row)
new_rows_df = pd.DataFrame(new_rows)
data = pd.concat([data, new_rows_df], ignore_index=True)
data = data.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)
for user_id in data['user_id'].unique():
    user_data = data[data['user_id'] == user_id].sort_values(by='timestamp')
    for index, row in user_data.iterrows():
        if row['click_activities'] == 'Causes We Support' or 'Insurance Products' in row['click_activities']:
            # if not user_data.loc[:index - 1, 'click_activities'].eq('ad click from eNewsletter').any():
                new_row = {
                    'user_id': user_id,
                    'table_from': '[Web_Clicks]',
                    'click_action': 'ad click from eNewsletter',
                    'timestamp': row['timestamp'] - pd.Timedelta(minutes=1),
                    'click_activities': 'ad click from eNewsletter'
                }
                new_rows.append(new_row)

new_rows_df = pd.DataFrame(new_rows)
data = pd.concat([data, new_rows_df], ignore_index=True)
data = data.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)

## define the starting point
new_rows = []
for user_id in data['user_id'].unique():
    user_data = data[data['user_id'] == user_id].sort_values(by='timestamp')
    first_action = user_data.iloc[0]
    if first_action['table_from'] != '[User_Login_Tracking]'and first_action['click_activities'] not in ['ad click from POLICY-Wizard website', 'carrier']:
        new_row = {
            'user_id': user_id,
            'table_from': '[User_Login_Tracking]',
            'click_action': 'login from mocaller',
            'timestamp': first_action['timestamp'] - pd.Timedelta(minutes=1),
            'click_activities': 'POLICY-Wizard: from eNewsletter'
        }
        new_rows.append(new_row)
    elif first_action['click_activities'] in ['ad click from POLICY-Wizard website', 'carrier']:
        new_row = {
            'user_id': user_id,
            'table_from': '[User_Login_Tracking]',
            'click_action': 'login from login_page',
            'timestamp': row['timestamp'] - pd.Timedelta(minutes=1),
            'click_activities': 'POLICY-Wizard: login page'
        }
        new_rows.append(new_row)
        same_time_rows = user_data[user_data['timestamp'] == first_action['timestamp']]
        for index, row in same_time_rows.iterrows():
            new_rows.append({
                'user_id': user_id,
                'table_from': '[User_Login_Tracking]',
                'click_action': 'login from login_page',
                'timestamp': row['timestamp'] - pd.Timedelta(minutes=1),
                'click_activities': 'POLICY-Wizard: login page'
            })
new_rows_df = pd.DataFrame(new_rows)

data = pd.concat([data, new_rows_df], ignore_index=True)

data = data.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)
#session added:
data['session'] = 0

# Assign sessions to each user
for user_id in data['user_id'].unique():
    user_data = data[data['user_id'] == user_id].sort_values(by='timestamp')
    session = 1
    data.loc[user_data.index[0], 'session'] = session  # Assign session to the first action
    for idx in range(1, len(user_data)):
        if user_data.iloc[idx]['table_from'] == '[User_Login_Tracking]':
            session += 1  # New login found, increment session
        data.loc[user_data.index[idx], 'session'] = session  # Assign session to the current action

data = data.sort_values(by=['user_id', 'timestamp', 'session']).reset_index(drop=True)


##Visulization
all_paths = []
stage_counts = {}

for user_id, group in data.groupby(['user_id', 'session']):
    group = group.reset_index(drop=True)
    for stage in range(len(group)):
        if stage == 0:
            start_point = group.loc[stage, 'click_activities']
            if start_point not in stage_counts:
                stage_counts[start_point] = 0
            stage_counts[start_point] += 1
        else:
            source = group.loc[stage - 1, 'click_activities']
            target = group.loc[stage, 'click_activities']
            all_paths.append((f'Stage {stage}: {source}', f'Stage {stage + 1}: {target}'))
            if f'Stage {stage + 1} : {target}' not in stage_counts:
              stage_counts[f'Stage {stage + 1}: {target}'] = 0
            stage_counts[f'Stage {stage + 1}: {target}'] += 1

path_df = pd.DataFrame(all_paths, columns=['source', 'target'])


transition_counts = path_df.groupby(['source', 'target']).size().reset_index(name='count')


all_nodes = list(pd.concat([transition_counts['source'], transition_counts['target']]).unique())
node_map = {label: i for i, label in enumerate(all_nodes)}


transition_counts['source_index'] = transition_counts['source'].map(node_map)
transition_counts['target_index'] = transition_counts['target'].map(node_map)


fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=['']*len(all_nodes),
        hovertemplate='%{customdata}  total value %{value}<extra></extra>',
        customdata=all_nodes
    ),
    link=dict(
        source=transition_counts['source_index'],
        target=transition_counts['target_index'],
        value=transition_counts['count'],
        color="grey",
        customdata=[f'{source} -> {target}' for source, target in zip(transition_counts['source'], transition_counts['target'])],
        hovertemplate='Source: %{customdata}<br>Value: %{value}<extra></extra>' # Hover Text
    )
))

fig.update_layout(
    title_text="User Behavior Flow",
    font=dict(
        size=12,
        color='black'
    ),
    width=2000,
    height=1000
)

fig.show()


[NbConvertApp] WARNING | pattern 'behavior_flow.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
 

In [19]:
from google.colab import drive
drive.mount('/content/drive')
!jupyter nbconvert --to html /content/drive/My Drive/Colab Notebooks/behavior_flow.ipynb

ValueError: mount failed: timeout during initial read of root folder; for more info: https://research.google.com/colaboratory/faq.html#drive-timeout

In [15]:
import os
notebook_path = os.path.abspath("behavior_flow.ipynb")
print(notebook_path)

/content/behavior_flow.ipynb
